## Testing and timing of `moarchiving.BiobjectiveNondominatedSortedList`

In [1]:
import doctest
import moarchiving
# reload(moarchiving)
NA = moarchiving.BiobjectiveNondominatedSortedList
doctest.testmod(moarchiving)
# NA.make_expensive_asserts = True

TestResults(failed=0, attempted=44)

In [2]:
import numpy as np
def nondom_arch(n):
    return np.abs(np.linspace(-1, 1, 2 * n).reshape(2, n).T).tolist()
# nondom_arch(3)

In [3]:
import fractions
id_ = lambda x: x
rg = np.arange(0.1, 1000)

### Timing of `Fraction`

In [4]:
%timeit [id_(i) for i in rg]

10000 loops, best of 3: 124 µs per loop


In [5]:
%timeit [float(i) for i in rg]

10000 loops, best of 3: 170 µs per loop


In [6]:
%timeit [fractions.Fraction(i) for i in rg]

100 loops, best of 3: 2.7 ms per loop


### Various

In [7]:
a = moarchiving.BiobjectiveNondominatedSortedList(
    [[-0.749, -1.188], [-0.557, 1.1076],
     [0.2454, 0.4724], [-1.146, -0.110]], [10, 10])
a._asserts()
a

[[-1.146, -0.11], [-0.749, -1.188]]

In [8]:
a.dominators([1, 3]) == a

True

In [9]:
a.add([-1, -3])  # return index where the element was added

1

In [10]:
a

[[-1.146, -0.11], [-1, -3]]

In [11]:
a.add([-1.5, 44])

In [12]:
a

[[-1.146, -0.11], [-1, -3]]

In [13]:
a.dominates(a[0])

True

In [14]:
a.dominates([-1.2, 1])

False

In [15]:
a._asserts()

In [16]:
b = NA(a)
print(b.merge([[-1.2, 1]]))
print(a.add_list([[-1.2, 1]]))
assert b == a
print(a.merge(b))

1
1
0


In [17]:
r = np.random.randn(100_000, 2).tolist()
r2 = sorted(np.random.randn(200, 2).tolist())
assert NA(r).add_list(r2) == NA(r).merge(r2)

In [18]:
%%timeit a = NA(r)
a.add_list(r2)

1000 loops, best of 3: 396 µs per loop


In [19]:
%%timeit a = NA(r)
a.merge(r2)

1000 loops, best of 3: 313 µs per loop


## Timing of initialization

In [20]:
%timeit nondom_arch(1_000)
%timeit nondom_arch(10_000)
%timeit nondom_arch(100_000)  # just checking baseline

The slowest run took 5.48 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 95.7 µs per loop
1000 loops, best of 3: 873 µs per loop
100 loops, best of 3: 16.4 ms per loop


In [21]:
%timeit NA(nondom_arch(1_000))
%timeit NA(nondom_arch(10_000))
%timeit NA(nondom_arch(100_000))  # nondom_arch itself takes about 25%

1000 loops, best of 3: 1.07 ms per loop
100 loops, best of 3: 10.2 ms per loop
10 loops, best of 3: 112 ms per loop


In [22]:
randars = {}  # prepare input lists
for n in [1_000, 10_000, 100_000]:
    randars[n] = np.random.rand(n, 2).tolist()
len(NA(nondom_arch(100_000))), len(NA(randars[100_000]))

(100000, 11)

In [23]:
%timeit NA(randars[1_000])
%timeit NA(randars[10_000])
%timeit NA(randars[100_000])

1000 loops, best of 3: 730 µs per loop
100 loops, best of 3: 8.78 ms per loop
10 loops, best of 3: 181 ms per loop


In [24]:
%timeit sorted(nondom_arch(1_000))
%timeit sorted(nondom_arch(10_000))
%timeit sorted(nondom_arch(100_000))

10000 loops, best of 3: 130 µs per loop
1000 loops, best of 3: 1.2 ms per loop
10 loops, best of 3: 21 ms per loop


In [25]:
%timeit sorted(randars[1_000])
%timeit sorted(randars[10_000])
%timeit sorted(randars[100_000])

1000 loops, best of 3: 314 µs per loop
100 loops, best of 3: 4.97 ms per loop
10 loops, best of 3: 108 ms per loop


In [26]:
%timeit list(randars[1_000])
%timeit list(randars[10_000])
%timeit list(randars[100_000])

The slowest run took 4.14 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.38 µs per loop
10000 loops, best of 3: 34.8 µs per loop
1000 loops, best of 3: 957 µs per loop


### Summary with 1e5 data
```
   1 ms `list` 
  22 ms `sorted` on sorted list
 130 ms `sorted` on unsorted list
 110 ms archive on sorted nondominated list
 190 ms archive on list which needs pruning (was 1300ms)
```

## Timing of `add`

In [27]:
%%timeit a = NA(nondom_arch(1_000))
for i in range(1000):
    a.add([ai - 1e-4 for ai in a[np.random.randint(len(a))]])
len(a)

100 loops, best of 3: 7.29 ms per loop


In [28]:
%%timeit a = NA(nondom_arch(10_000))
for i in range(1000):
    a.add([ai - 1e-4 for ai in a[np.random.randint(len(a))]])
len(a)

100 loops, best of 3: 7.67 ms per loop


In [29]:
%%timeit a = NA(nondom_arch(100_000))
for i in range(1000):
    a.add([ai - 1e-4 for ai in a[np.random.randint(len(a))]])
len(a)  # deletion kicks in and makes it 20 times slower if implemented with pop

10 loops, best of 3: 18.4 ms per loop


In [30]:
%%timeit a = NA(nondom_arch(100_000))
for i in range(1000):
    a.add([ai - 1e-8 for ai in a[np.random.randint(len(a))]])
len(a) # no deletion has taken place

100 loops, best of 3: 10.4 ms per loop


In [31]:
%%timeit a = NA(nondom_arch(1_000_000))
for i in range(1000):
    a.add([ai - 1e-4 for ai in a[np.random.randint(len(a))]])
len(a)  # deletion kicks in

1 loop, best of 3: 267 ms per loop


In [32]:
%%timeit a = NA(nondom_arch(1_000_000))
for i in range(1000):
    a.add([ai - 1e-8 for ai in a[np.random.randint(len(a))]])
len(a)

100 loops, best of 3: 12.4 ms per loop


## Timing of Hypervolume computation

In [33]:
%timeit a = NA(nondom_arch(1_000), [5, 5])  # takes 4 or 40x longer than without hypervolume computation

10 loops, best of 3: 27.6 ms per loop


In [34]:
%timeit a = NA(nondom_arch(10_000), [5, 5])

1 loop, best of 3: 298 ms per loop


In [35]:
%timeit a = NA(nondom_arch(100_000), [5, 5])  # takes 3x longer than without hypervolume computation

1 loop, best of 3: 2.94 s per loop


In [36]:
%%timeit a = NA(nondom_arch(1_000), [5, 5])
a.hypervolume

The slowest run took 9.09 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 218 ns per loop


In [37]:
%%timeit a = NA(nondom_arch(10_000), [5, 5])
a.hypervolume

The slowest run took 11.48 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 219 ns per loop


In [38]:
%%timeit a = NA(nondom_arch(100_000), [5, 5]) 
a.hypervolume

The slowest run took 12.51 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 211 ns per loop


In [39]:
NA.hypervolume_computation_float_type = float

In [40]:
%timeit a = NA(nondom_arch(1_000), [5, 5])  # takes 4 or 40x longer than without hypervolume computation

100 loops, best of 3: 10.9 ms per loop


In [41]:
NA.hypervolume_final_float_type = float
NA.hypervolume_computation_float_type = float

In [42]:
%timeit a = NA(nondom_arch(1_000), [5, 5])  # takes 4 or 40x longer than without hypervolume computation

100 loops, best of 3: 3.76 ms per loop
